# BEECN Location Visualization

In [25]:

import os
import pandas as pd
import folium

neigh_geo = 'https://opendata.arcgis.com/datasets/9f50a605cf4945259b983fa35c993fe9_125.geojson'
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')

# neighbor_file = os.path.join(data_dir, 'neighborhood_population.csv')
neighbor_file = os.path.join(data_dir, 'Census_2010_Data_Cleanedup_Neighborhoods2.csv')

# Way Jupyter Notebook and local machine read in files is different. Use this 
if os.path.exists(neighbor_file):
    neighbor_df = pd.read_csv(neighbor_file)
else:
    data_dir = os.path.join(os.getcwd(), 'pybeecn/data')
    neighbor_file = os.path.join(data_dir, 'Census_2010_Data_Cleanedup_Neighborhoods2.csv')
    neighbor_df = pd.read_csv(neighbor_file)

# for i in range(len(neighbor_df.columns)):
#     # print(i)
#     print(neighbor_df.columns[i])
    
m = folium.Map(location=[45.5236, -122.6750])

# population = folium.Choropleth(
#                                geo_data=neigh_geo,
#                                data=neighbor_df,
#                                columns=['OBJECTID', 'Total'],
#                                key_on='feature.properties.OBJECTID',
#                                fill_color='YlGn',
#                                fill_opacity=0.9,
#                                line_opacity=0.2,
#                                legend_name='Neighborhood Population Size',
#                                highlight=True,
#                                name='2010 Population',
#                                show=True
# ).add_to(m)

# folium.GeoJson(
#     neigh_geo,
#     tooltip=folium.features.GeoJsonTooltip(fields=['NAME'],
#                                            localize=True)
# ).add_to(population.geojson)


# m.save('population.html')

# plots_dir = os.path.join(os.path.dirname(os.getcwd()), 'plots')
# if os.path.exists(plots_dir):
#     m.save(os.path.join(plots_dir), 'map.html')
# else:
#     plots_dir = os.path.join(os.getcwd(), 'pybeecn/plots')
#     m.save(os.path.join(plots_dir), 'map.html')
# print(plots_dir)
white = folium.Choropleth(
                          geo_data=neigh_geo,
                          data=neighbor_df,
                          columns=['OBJECTID', 'White alone (NHoL)'],
                          key_on='feature.properties.OBJECTID',
                          fill_color='GnBu',
                          fill_opacity=1,
                          line_opacity=0.2,
                          legend_name='Neighborhood Population Size',
                          highlight=True,
                          name='2010 White Population',
                          show=False
).add_to(m)

# folium.GeoJson(
#     neigh_geo,
#     tooltip=folium.features.GeoJsonTooltip(fields=['NAME'],
#                                            localize=True)
# ).add_to(white.geojson)

black = folium.Choropleth(
                          geo_data=neigh_geo,
                          data=neighbor_df,
                          columns=['OBJECTID', 'Black or African American alone (NH0L)'],
                          key_on='feature.properties.OBJECTID',
                          fill_color='OrRd',
                          fill_opacity=1,
                          line_opacity=0.2,
                          legend_name='Neighborhood Population Size',
                          highlight=True,
                          name='2010 Black Population',
                          show=False
).add_to(m)

# todo: Having a problem loading more than two layers into Choropleth. 

# indian = folium.Choropleth(
#                           geo_data=neigh_geo,
#                           data=neighbor_df,
#                           columns=['OBJECTID', 'American Indian and Alaska Native alone (NH0L)'],
#                           key_on='feature.properties.OBJECTID',
#                           fill_color='OrRd',
#                           fill_opacity=1,
#                           line_opacity=0.2,
#                           legend_name='Neighborhood Population Size',
#                           highlight=True,
#                           name='2010 NA/Alaskan Population',
#                           show=False
# ).add_to(m)
folium.LayerControl().add_to(m)
m


In [6]:
import pandas as pd
import numpy as np
import geopandas as gpd
url_beecns = 'https://opendata.arcgis.com/datasets/6e6185533d5447deb8b7204c27e1858e_92.geojson'
beecn_geo = gpd.read_file(url_beecns)
beecn_geo = pd.DataFrame(beecn_geo)

beecn_geo['geometry'] = beecn_geo['geometry'].map(lambda x: str(x).lstrip('POINT (').rstrip(')'))
lat = []
lon = []

for row in beecn_geo['geometry']:
    try:
        lon.append(row.split(' ')[0])
        lat.append(row.split(' ')[1])
    except:
        lon.append(np.NaN)
        lat.append(np.NaN)

beecn_geo['latitude'] = lat
beecn_geo['longitude'] = lon
print(beecn_geo.head())

   OBJECTID                          SITE_NAME  \
0         1         Parkrose High School Field   
1         2  Ainsworth Elementary School Field   
2         3    Chapman Elementary School Field   
3         4   Sunstone Montessori School Field   
4         5                     Lair Hill Park   

                              LOCATION SITE_OWNER    X_COORD   Y_COORD MAP_ID  \
0               12003 NE Shaver Street        PPS  7678990.0  694114.0  NE-19   
1     SW Elm Street and SW 21st Avenue        PPS  7638110.0  679716.0  SW-24   
2  NW Quimby Street and NW 27th Avenue        PPS  7636770.0  688215.0   NW-3   
3               6318 SW Corbett Street        PPS  7643970.0  667980.0  SW-29   
4    SW 2nd Aveue and SW Porter Street       PP&R  7643160.0  676494.0  SW-25   

  BARRIER_ID                              geometry           latitude  \
0          L  -122.5422587229825 45.55243515646136  45.55243515646136   
1          F  -122.7002495885055 45.51001783951922  45.51001783951

## Generate a Map

Generate a map of the BEECN Locations with a 1600 meter radius around each site. The map also shows the boundaries of each Portland Neighborhood. 

In [ ]:
latitude = np.array(beecn_geo['latitude'])
latitude = latitude.astype(float)
longitude = np.array(beecn_geo['longitude'])
longitude = longitude.astype(float)
m = folium.Map(location=[45.5236, -122.6750])
beecn_site = beecn_geo['SITE_NAME']
address = beecn_geo['LOCATION']

location = zip(latitude, longitude)

count = 0
for i in location:
    folium.Marker(i, popup=address[count], icon= folium.Icon(icon='medkit', prefix='fa'), tooltip=beecn_site[count]).add_to(m)
    folium.Circle(i, radius=1600).add_to(m)
    count += 1
m

NameError: name 'beecn_geo' is not defined